In [ ]:
from ultralytics import YOLO
import cv2
import os
import numpy as np

# Carregar o modelo
# Load the model
model = YOLO("yolov8n-pose.pt")  # Carregar um modelo oficial / Load an official model
# model = YOLO("path/to/best.pt")  # Carregar um modelo personalizado / Load a custom model

# Definir o diretório de saída
# Define the output directory
output_dir = 'yolov8/granite-detection/runs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Definir o caminho do vídeo de entrada e saída
# Define the input and output video paths
input_video_path = os.path.join(output_dir, 'input_video.mp4')
output_video_path = os.path.join(output_dir, 'output_video.mp4')

# Abrir o vídeo de entrada
# Open the input video
cap = cv2.VideoCapture(input_video_path)

# Verificar se o vídeo foi aberto corretamente
# Check if the video was opened correctly
if not cap.isOpened():
    print(f"Erro ao abrir o vídeo: {input_video_path}")
    exit(1)

# Obter informações do vídeo
# Get video information
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Definir a fração do vídeo a ser processada (10% iniciais)
# Define the fraction of the video to be processed (initial 10%)
fraction_to_process = 0.1
frames_to_process = int(total_frames * fraction_to_process)

# Definir o codec e criar o objeto de escrita de vídeo
# Define the codec and create the video writer object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Você pode usar 'XVID', 'MJPG', etc. / You can use 'XVID', 'MJPG', etc.
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# IDs dos keypoints para o ombro direito, quadril direito e pés
# Keypoint IDs for the right shoulder, right hip, and feet
right_shoulder_id = 6  # Ajuste conforme necessário / Adjust as needed
right_hip_id = 12  # Ajuste conforme necessário / Adjust as needed
right_foot_id = 16  # Ajuste conforme necessário / Adjust as needed
left_foot_id = 15  # Ajuste conforme necessário / Adjust as needed

# Listas para armazenar as posições do ombro direito, quadril direito e pés
# Lists to store the positions of the right shoulder, right hip, and feet
right_shoulder_positions = []
right_hip_positions = []
right_foot_positions = []
left_foot_positions = []

# Definir confiança mínima
# Define minimum confidence
min_confidence = 0.5

# Definir um limite mínimo de movimento (em pixels)
# Define a minimum movement threshold (in pixels)
min_movement_threshold = 5

# Função para contar giros
# Function to count spins
def count_spins(positions):
    spins = 0
    for i in range(1, len(positions) - 1):
        if abs(positions[i-1][0] - positions[i][0]) > min_movement_threshold and \
           abs(positions[i+1][0] - positions[i][0]) > min_movement_threshold:
            spins += 1
    return spins

# Função para classificar movimentos
# Function to classify movements
def classify_movement(spins):
    if spins == 1:
        return "Movimento A"
    elif spins == 1.5:
        return "Movimento B"
    elif spins == 2:
        return "Movimento C"
    else:
        return "Movimento Desconhecido"

# Função para calcular a velocidade dos pés
# Function to calculate foot speed
def calculate_speed(foot_positions, fps):
    speeds = []
    for i in range(1, len(foot_positions)):
        distance = np.linalg.norm(np.array(foot_positions[i]) - np.array(foot_positions[i-1]))
        speed = distance * fps
        speeds.append(speed)
    return speeds

# Função para determinar se a pessoa está correndo ou andando
# Function to determine if the person is running or walking
def detect_running_walking(speeds, threshold=5.0):
    running = sum(speed > threshold for speed in speeds)
    walking = len(speeds) - running
    return running, walking

# Variáveis para contagem de saltos, giros e passos
# Variables for counting jumps, spins, and steps
in_air = False
jumps = 0
spins_per_jump = []
total_air_spins = 0
total_ground_spins = 0
steps = 0
movement_classes = []

# Contadores para cada tipo de movimento
# Counters for each type of movement
movement_counts = {"Movimento A": 0, "Movimento B": 0, "Movimento C": 0, "Movimento Desconhecido": 0}

# Verificar movimento significativo entre frames
# Check for significant movement between frames
def is_significant_movement(position1, position2, threshold=min_movement_threshold):
    return np.linalg.norm(np.array(position1) - np.array(position2)) > threshold

# Processar cada frame do vídeo
# Process each frame of the video
for frame_idx in range(frames_to_process):
    ret, frame = cap.read()
    if not ret:
        break

    # Fazer predição com o modelo
    # Make prediction with the model
    results = model(frame)

    # Renderizar resultados na imagem e coletar posições do ombro, quadril direito e pés
    # Render results on the image and collect positions of the shoulder, right hip, and feet
    for result in results:
        if hasattr(result, 'keypoints') and result.keypoints is not None and len(result.boxes) > 0:
            keypoints = result.keypoints.data[0].cpu().numpy()  # Assumindo que os keypoints estão no formato esperado / Assuming keypoints are in the expected format
            confidence = result.boxes.conf[0].item()  # Acessar a confiança da detecção / Access detection confidence
            if confidence >= min_confidence:
                print(f"Frame {frame_idx} Keypoints detectados:")
                for idx, (x, y, c) in enumerate(keypoints):
                    print(f"Keypoint {idx}: ({x}, {y}), Confiança: {c}")
                if len(keypoints) > max(right_shoulder_id, right_hip_id, right_foot_id, left_foot_id):
                    right_shoulder = keypoints[right_shoulder_id]
                    right_hip = keypoints[right_hip_id]
                    right_foot = keypoints[right_foot_id]
                    left_foot = keypoints[left_foot_id]

                    if len(right_shoulder_positions) > 0:
                        if is_significant_movement(right_shoulder_positions[-1], (right_shoulder[0], right_shoulder[1])):
                            right_shoulder_positions.append((right_shoulder[0], right_shoulder[1]))
                    
                    if len(right_hip_positions) > 0:
                        if is_significant_movement(right_hip_positions[-1], (right_hip[0], right_hip[1])):
                            right_hip_positions.append((right_hip[0], right_hip[1]))
                    
                    if len(right_foot_positions) > 0:
                        if is_significant_movement(right_foot_positions[-1], (right_foot[0], right_foot[1])):
                            right_foot_positions.append((right_foot[0], right_foot[1]))
                            steps += 1  # Contar apenas se houver movimento significativo / Count only if there is significant movement
                    
                    if len(left_foot_positions) > 0:
                        if is_significant_movement(left_foot_positions[-1], (left_foot[0], left_foot[1])):
                            left_foot_positions.append((left_foot[0], left_foot[1]))

                    print(f"Right Hip Position: {right_hip}")

                    # Detectar saltos baseados na posição vertical do quadril
                    # Detect jumps based on the vertical position of the hip
                    if right_hip[1] < height * 0.5:  # Ajustar este valor conforme necessário para detectar saltos / Adjust this value as needed to detect jumps
                        if not in_air:
                            in_air = True
                            jumps += 1
                            spins_per_jump.append(0)
                            print(f"Salto detectado no frame {frame_idx}")
                    else:
                        if in_air:
                            in_air = False
                            spins = count_spins(right_shoulder_positions[-fps:])
                            spins_per_jump[-1] = spins
                            total_air_spins += spins
                            movement_class = classify_movement(spins)
                            movement_classes.append(movement_class)
                            movement_counts[movement_class] += 1

                    # Contar giros no solo
                    # Count ground spins
                    if not in_air:
                        ground_spins = count_spins(right_shoulder_positions[-fps:])
                        total_ground_spins += ground_spins

    # Renderizar resultados na imagem
    # Render results on the image
    if len(results) > 0:
        rendered_frame = results[0].plot()
    else:
        rendered_frame = frame
    
    # Adicionar contadores na tela
    # Add counters on the screen
    font_scale = 0.8
    thickness = 2
    cv2.putText(rendered_frame, f"Saltos: {jumps}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), thickness, cv2.LINE_AA)
    cv2.putText(rendered_frame, f"Giros no Ar: {total_air_spins}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), thickness, cv2.LINE_AA)
    cv2.putText(rendered_frame, f"Giros no Solo: {total_ground_spins}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), thickness, cv2.LINE_AA)
    cv2.putText(rendered_frame, f"Passos: {steps}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), thickness, cv2.LINE_AA)
    
    # Adicionar contadores de movimentos na tela
    # Add movement counters on the screen
    y_offset = 150
    for movement, count in movement_counts.items():
        cv2.putText(rendered_frame, f"{movement}: {count}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), thickness, cv2.LINE_AA)
        y_offset += 30

    # Escrever o frame processado no vídeo de saída
    # Write the processed frame to the output video
    out.write(rendered_frame)

# Calcular a velocidade dos pés e detectar corrida ou caminhada
# Calculate foot speed and detect running or walking
right_foot_speeds = calculate_speed(right_foot_positions, fps)
left_foot_speeds = calculate_speed(left_foot_positions, fps)

running_right, walking_right = detect_running_walking(right_foot_speeds)
running_left, walking_left = detect_running_walking(left_foot_speeds)

total_running = running_right + running_left
total_walking = walking_right + walking_left

# Liberar os objetos de captura e escrita de vídeo
# Release the video capture and writer objects
cap.release()
out.release()

# Exibir os resultados finais
# Display final results
print(f"Total de saltos detectados: {jumps}")
print(f"Giros no ar detectados por salto: {spins_per_jump}")
print(f"Giros no solo: {total_ground_spins}")
print(f"Contagem de movimentos: {movement_counts}")
print(f"Passos detectados: {steps}")
print(f"Correndo: {total_running}, Andando: {total_walking}")
print(f"Vídeo salvo em: {output_video_path}")
